In [127]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import pandas as pd
import talib as ta
import importlib
import os
import platform
import psutil
import numpy as np
import BacktestingStrategies.Optimization as opt

#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
#https://www.morningstar.co.jp/etf_foreign/search.jsp
# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-10)
end = datetime.date.today()
# data = web.DataReader('1570.JP', 'stooq', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
# data = web.DataReader('1357.JP', 'stooq', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
data = web.DataReader('2558.JP', 'stooq', start, end)  #MAXIS 米国株式(S&P500)上場投信
data = data.astype("double")
print(data)

initial = 1000_000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

df_returns = pd.DataFrame()

               Open     High      Low    Close   Volume
Date                                                   
2022-10-17  15440.0  15495.0  15385.0  15490.0  48699.0
2022-10-14  15570.0  15710.0  15525.0  15680.0  75963.0
2022-10-13  15170.0  15195.0  15135.0  15145.0  34460.0
2022-10-12  15110.0  15220.0  15090.0  15210.0  35254.0
2022-10-11  15205.0  15215.0  15080.0  15090.0  94267.0
...             ...      ...      ...      ...      ...
2020-01-16  10400.0  10400.0  10320.0  10350.0   1391.0
2020-01-15  10400.0  10400.0  10300.0  10320.0   5119.0
2020-01-14  10390.0  10400.0  10350.0  10370.0   5739.0
2020-01-10  10300.0  10320.0  10280.0  10320.0   2502.0
2020-01-09  10180.0  10250.0  10180.0  10230.0   3389.0

[676 rows x 5 columns]


**Methods**

In [128]:
def execute_backtest(bt):
    output = bt.run()
    print(output)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

def execute_optimize(bt, opt_params):
    output = bt.optimize(**opt_params)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

**RSI**

In [129]:
import BacktestingStrategies.Prod_Strategy_Rsi as btstRsi
importlib.reload(btstRsi)
importlib.reload(opt)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

# # #最適化
# opt_params = {
#     "upper_bound": range(50, 85, 5),
#     "lower_bound": range(10, 45, 5),
#     "rsi_window": range(10, 30, 2),
#     "atr_window": range(10, 30, 2),
#     "atr_stoploss": range(1, 10, 1),
#     "atr_takeprofit": range(1, 10, 1),
#     "maximize": opt.optim_func,\
#     "method": 'grid',\
#     "max_tries": 1000}
# execute_optimize(bt, opt_params)

stop loss  2020-02-28 00:00:00  10029.401  9147.610837761693
RSI close 2020-04-21 00:00:00
RSI close 2020-07-07 00:00:00
RSI close 2020-10-15 00:00:00
RSI close 2020-11-13 00:00:00
RSI close 2021-07-09 00:00:00
RSI close 2021-07-28 00:00:00
take profit  2021-10-26 00:00:00  13677.3695  14859.06420804497
RSI close 2022-01-06 00:00:00
stop loss  2022-02-24 00:00:00  14908.43325  13849.07070630823
take profit  2022-03-29 00:00:00  13923.58225  16082.457249289408
RSI close 2022-06-06 00:00:00
RSI close 2022-07-25 00:00:00
Start                     2020-01-09 00:00:00
End                       2022-10-17 00:00:00
Duration                   1012 days 00:00:00
Exposure Time [%]                   34.319527
Equity Final [$]                1709176.67575
Equity Peak [$]                 1752076.67575
Return [%]                          70.917668
Buy & Hold Return [%]                 51.4174
Return (Ann.) [%]                   22.117687
Volatility (Ann.) [%]               17.176921
Sharpe Ratio    

C:\Users\javie\AppData\Local\Temp\ipykernel_21968\531229275.py:6: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,99,32,34,10029.40100,9100.0,-92010.69900,-0.092668,2020-02-27,2020-03-02,4 days
1,115,57,70,7868.75850,8460.0,67992.77250,0.075138,2020-04-03,2020-04-22,19 days
2,102,105,121,9526.92600,9680.0,15613.54800,0.016068,2020-06-16,2020-07-08,22 days
3,101,172,187,9798.26250,10550.0,75925.48750,0.076722,2020-09-24,2020-10-16,22 days
4,107,196,207,9888.70800,10850.0,102858.24400,0.097211,2020-10-29,2020-11-16,18 days
5,87,353,367,13406.03300,13800.0,34275.12900,0.029387,2021-06-22,2021-07-12,20 days
6,87,374,378,13697.46850,13850.0,13270.24050,0.011136,2021-07-21,2021-07-29,8 days
7,89,415,439,13677.36950,15010.0,118604.11450,0.097433,2021-09-22,2021-10-27,35 days
8,88,466,487,15084.29950,15640.0,48901.64400,0.036840,2021-12-07,2022-01-07,31 days
9,92,495,519,14908.43325,14150.0,-69775.85900,-0.050873,2022-01-20,2022-02-25,36 days
